### Iterating Collections

인덱스별로 요소에 액세스할 수 있어 시퀀스 유형이 반복을 지원하는 방법을 확인했다.
우리는 `__get_item__` 방법을 구현하여 사용자 정의 시퀀스 유형을 작성할 수도 있다.

그러나 몇 가지 제한이 있습니다:

* 항목은 숫자로 인덱싱할 수 있어야 하며 인덱싱은 '0'부터 시작해야 합니다
* 집합과 같이 순서가 지정되지 않은 집합에는 사용할 수 없습니다

컬렉션을 반복할 경우 컬렉션의 **next** 항목을 요청할 수 있는 방법이 필요합니다.

만약 우리가 그렇게 할 수 있다면, 우리의 컬렉션은 색인화될 필요도 없고, 순서를 정할 필요도 없다(즉, 우리는 컨테이너에 있는 요소들의 상대적인 위치에 대한 개념이 필요하지 않다).

이것이 일반적으로 반복 가능한 것입니다. 반복 가능한 것은 집합에서 "다음" 요소를 반환하는 메소드를 제공합니다.
이 접근법은 시퀀스 유형 컬렉션뿐만 아니라 집합과 같은 순서가 지정되지 않은 컬렉션 유형에서도 동일하게 잘 작동합니다.

물론 **next**가 정렬되지 않은 컬렉션에서 항목을 반환하는 순서는 사전에 알려져 있지 않습니다. 예를 들어, 집합에 대해 반복할 때 다음과 같이 확인할 수 있습니다:

In [1]:
s = {'x', 'y', 'b', 'c', 'a'}
for item in s:
    print(item)

y
a
c
b
x


집합의 요소가 반환된 순서를 볼 수 있듯이, 집합에 요소를 추가한 순서와 일치하지 않습니다.

또한 인덱싱을 사용하여 집합의 요소에 액세스할 수 없습니다

In [2]:
s[0]

TypeError: 'set' object does not support indexing

### Rolling our own Next method

우리의 Iterable을 정의해보자

우리가 하고자 하는 것은 `__get_item__` 메서드 대신 `next` 메서드를 구현하는 컨테이너 유형의 클래스를 갖는 것이다.

`next`을 호출할 때마다 컬렉션의 다음 요소를 반환해야 하므로 반복에서 우리가 어디에 있는지를 어떻게든 추적해야 할 것이다.

`next`는 내장된 함수이기 때문에 앞에서 살펴본 바와 같이 대신 `next_`를 사용할 것이다.

In [3]:
class Squares:
    def __init__(self):
        self.i = 0
    
    def next_(self):
        result = self.i ** 2
        self.i += 1
        return result

In [4]:
sq = Squares()

In [5]:
sq.next_()

0

In [6]:
sq.next_()

1

In [7]:
sq.next_()

4

어떻게 처음부터 반복을 다시 시작할까요?

우리는 할 수 없다 - 우리는 `Squares`의 새로운 인스턴스를 만들어야 한다:

In [8]:
sq = Squares()

In [9]:
for i in range(10):
    print(sq.next_())

0
1
4
9
16
25
36
49
64
81


We even are able to iterate over the squares.

하지만 기본적으로 **무한**개의 항목이 있습니다.

우리는 컬렉션을 만들 때 길이를 지정하고 `next_()`가 컬렉션의 요소 수를 초과하면 예외를 발생시킴으로써 충분히 쉽게 수정할 수 있다. 우리는 `StopItation` 예외를 발생시킬 것이다. 이것은 파이썬이 특별히 이런 종류의 시나리오에 대해 제공하는 내장된 예외 이다!!

우리는 `len()` 함수를 지원하기 위해 `__len__` 메서드를 구현할 것이다:

In [10]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
    
    def next_(self):
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i ** 2
            self.i += 1
            return result           
        
    def __len__(self):
        return self.length

In [11]:
sq = Squares(3)

In [12]:
len(sq)

3

In [13]:
sq.next_()

0

In [14]:
sq.next_()

1

In [15]:
sq.next_()

4

In [16]:
sq.next_()

StopIteration: 

그래서 이제 우리는 본질적으로 시퀀스와 `__get_item__` 메서드를 사용한 방법과 매우 유사한 방식으로 컬렉션을 루프할 수 있다:

In [17]:
sq = Squares(5)
while True:
    try:
        print(sq.next_())
    except StopIteration:
        # reached end of iteration
        # stop looping
        break       

0
1
4
9
16


두가지 이슈가 있습니다.
첫번째는 Iterable `sq` 가 소진되었다는 것입니다. - 우리는 반복을 "다시시작"할 수 없습니다.

In [18]:
sq.next_()

StopIteration: 

두 번째 문제는 `for` 루프를 사용할 수 없다는 것이다 - 파이썬은 우리의 `next_()` 메서드에 대해 알지 못한다:

In [19]:
for i in Squares(10):
    print(i)

TypeError: 'Squares' object is not iterable

물론 `__getitem__` 방법이 있다면 모든 것이 다시 작동하겠지만 `__getitem__`은 시퀀스 유형이 있음을 의미한다는 것을 기억하십시오. 사각형이 실제로 시퀀스이긴 하지만, 반드시 시퀀스가 아닌 컨테이너를 만드는 보다 일반적인 방법을 알아보려고 합니다.

파이썬의 `len()` 함수 및 `__len__()` 메서드와 마찬가지로 파이썬에는 내장된 `next()` 함수가 있으며, 클래스에 `__next__()` 메서드가 있으면 이 함수를 호출한다.

이것을 보자:

In [20]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
    
    def __next__(self):
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i ** 2
            self.i += 1
            return result   
    
    def __len__(self):
        return self.length

In [21]:
sq = Squares(3)

In [22]:
next(sq)

0

In [23]:
next(sq)

1

In [24]:
next(sq)

4

In [25]:
next(sq)

StopIteration: 

그래서 그것은 좀 더 쉽게 타이핑할 수 있게 해준다. 우리가 이전에 썼던 루프는 지금 이렇게 보일 것이다:

In [26]:
sq = Squares(5)
while True:
    try:
        print(next(sq))
    except StopIteration:
        break  

0
1
4
9
16


이것은 Python이 이제 Squares의 인스턴스에서 반복할 수 있다는 것을 의미합니까?

In [27]:
for i in Squares(10):
    print(i)

TypeError: 'Squares' object is not iterable

아니요, Python은 여전히 우리 클래스를 참을 수 있는 컬렉션으로 인식하지 않습니다.

우리는 그곳에 가기 위해 조금 더 많은 일을 해야 한다.

우리는 또한 완전히 새로운 객체를 만들지 않고 반복을 "재설정"하는 방법을 살펴볼 필요가 있다.

기술적으로 우리의 `Squares` 클래스는 시퀀스 유형으로 구축될 수 있다는 것을 알게 될 것입니다. 그것은 매우 간단한 예에 불과했습니다.

대신, 임의의 숫자가 들어 있지만 특정 순서는 아닌 다른 집합을 만들어 봅시다.

In [28]:
import random

In [29]:
class RandomNumbers:
    def __init__(self, length, *, range_min=0, range_max=10):
        self.length = length
        self.range_min = range_min
        self.range_max = range_max
        self.num_requested = 0
        
    def __len__(self):
        return self.length
    
    def __next__(self):
        if self.num_requested >= self.length:
            raise StopIteration
        else:
            self.num_requested += 1
            return random.randint(self.range_min, self.range_max)

이제 이 개체의 인스턴스를 반복할 수 있습니다:

In [30]:
numbers = RandomNumbers(10)

In [31]:
len(numbers)

10

In [32]:
while True:
    try:
        print(next(numbers))
    except StopIteration:
        break

8
9
3
10
10
9
0
10
10
1


우리는 여전히 `for` 루프를 사용할 수 없으며, 반복을 `재시작`하려면 매번 새로운 객체를 생성해야 한다.

In [33]:
numbers = RandomNumbers(10)

In [34]:
for item in numbers:
    print(item)

TypeError: 'RandomNumbers' object is not iterable